In [ ]:
!tar -xvf ./data/hebtb.tar.gz

In [1]:
import pandas as pd
import numpy as np

## Files

In [2]:
filepath_spmrl_dev = './data/spmrl-treebank/dev_hebtb-gold.conll'
filepath_spmrl_train = './data/spmrl-treebank/train_hebtb-gold.conll'
filepath_spmrl_test = './data/spmrl-treebank/test_hebtb-gold.conll'

filepath_ud_dev = './data/ud-treebank/he_htb-ud-dev.conllu'
filepath_ud_train = './data/ud-treebank/he_htb-ud-train.conllu'
filepath_ud_test = './data/ud-treebank/he_htb-ud-test.conllu'


In [3]:
def suit_for_pandas(filepath):
    treebank = []
    source = open(filepath, 'r')
    for line in source.readlines():
        if len(line.split('\t')) == 10:
            treebank.append(tuple(line.strip().split('\t')))
        elif len(line.split('\t')) == 1:
            treebank.append((line.strip(), '', '', '', '', '', '', '', '', ''))
    if 'spmrl' in filepath:
        return pd.DataFrame(data = treebank, columns=['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC'])
    else:
        return pd.DataFrame(data = treebank, columns=['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC'])



In [4]:
ud_dev = suit_for_pandas(filepath_ud_dev)

In [5]:
spmrl_dev = suit_for_pandas(filepath_spmrl_dev)

In [16]:
ud_dev

,ID,FORM,LEMMA,UPOS,XPOS,FEATS,HEAD,DEPREL,DEPS,MISC,sent_id
0,# sent_id = 1,,,,,,,,,,
1,# text = עשרות אנשים מגיעים מתאילנד לישראל כשה...,,,,,,,,,,1
2,1,עשרות,עשרות,NUM,NUM,Definite=Cons|Gender=Fem|Number=Plur,2,nummod,_,_,1
3,2,אנשים,איש,NOUN,NOUN,Gender=Masc|Number=Plur,3,nsubj,_,_,1
4,3,מגיעים,הגיע,VERB,VERB,Gender=Masc|HebBinyan=HIFIL|Number=Plur|Person...,0,root,_,_,1
5,4-5,מתאילנד,_,_,_,_,_,_,_,_,1
6,4,מ,מ,ADP,ADP,_,5,case,_,_,1
7,5,תאילנד,תאילנד,PROPN,PROPN,_,3,obl,_,_,1
8,6-7,לישראל,_,_,_,_,_,_,_,_,1
9,6,ל,ל,ADP,ADP,_,7,case,_,_,1


In [12]:
spmrl_dev

,ID,FORM,LEMMA,UPOS,XPOS,FEATS,HEAD,DEPREL,DEPS,MISC,sent_id
0,1,עשרות,עשר,CDT,CDT,gen=F|num=P,2,num,_,_,
1,2,אנשים,איש,NN,NN,gen=M|num=P,3,subj,_,_,1
2,3,מגיעים,הגיע,BN,BN,gen=M|num=P|per=A,0,ROOT,_,_,1
3,4,מ,מ,PREPOSITION,PREPOSITION,_,3,prepmod,_,_,1
4,5,תאילנד,תאילנד,NNP,NNP,_,4,pobj,_,_,1
5,6,ל,ל,PREPOSITION,PREPOSITION,_,3,prepmod,_,_,1
6,7,ישראל,ישראל,NNP,NNP,_,6,pobj,_,_,1
7,8,כש,כש,TEMP,TEMP,_,3,comp,_,_,1
8,9,הם,הוא,PRP,PRP,gen=M|num=P|per=3,10,subj,_,_,1
9,10,נרשמים,נרשם,BN,BN,gen=M|num=P|per=A,14,conj,_,_,1


In [6]:
spmrl_dev['sent_id'] = ''
# def intify(x):
#     if x.isdigit():
#         return int(x)
    
spmrl_dev['ID'] = spmrl_dev['ID'].apply(lambda x: int(x) if x.isdigit() else 0)

In [7]:
ud_dev['sent_id'] = ''

In [17]:
spmrl_dev[spmrl_dev['sent_id'] == 484]

,ID,FORM,LEMMA,UPOS,XPOS,FEATS,HEAD,DEPREL,DEPS,MISC,sent_id
11527,0,,,,,,,,,,484
11528,1,מול,מול,IN,IN,_,10,prepmod,_,_,484
11529,2,בית,בית,NN,NN,gen=M|num=S,1,pobj,_,_,484
11530,3,מספר,מספר,NN,NN,gen=M|num=S,2,nn,_,_,484
11531,4,3,_,CD,CD,_,3,nn,_,_,484
11532,5,ב,ב,PREPOSITION,PREPOSITION,_,2,prepmod,_,_,484
11533,6,רחוב,רחוב,NNT,NNT,gen=M|num=S,5,pobj,_,_,484
11534,7,גבעת,גבעה,NNT,NNT,gen=F|num=S,6,gobj,_,_,484
11535,8,שאול,שאול,NNP,NNP,_,7,gobj,_,_,484
11536,9,אני,הוא,PRP,PRP,gen=F|gen=M|num=S|per=1,10,subj,_,_,484


In [ ]:
ud_dev[ud_dev['sent_id'] == 98]

In [8]:
def sentence_id(df, tb):
    if tb=='ud':
        sent_id = 0
        for i, row in df.iterrows():
            if '# sent_id' in row['ID']:
                sent_id += 1
            else:
                df.at[i, 'sent_id'] = sent_id
    elif tb=='spmrl':
        sent_id = 1
        for i, row in df.iterrows():
            if row['ID'] == '':
                continue
            
            try:
#                 if int(df.at[i, 'ID']) > int(df.at[i-1, 'ID']):
                if df.loc[i]['ID'] > df.loc[i-1]['ID']:
                    df.at[i, 'sent_id'] = sent_id
                elif type(df.loc[i-1]['ID']) == str:
                    continue
                else:
                    sent_id += 1
                    df.at[i, 'sent_id'] = sent_id
            except KeyError as e:
                print('somehing', e)
#                 df.at[i, 'sent_id'] = int(sent_id)
            except TypeError as e:
                print(df.loc[i]['ID'],df.loc[i-1]['ID'], e)

In [ ]:
int(spmrl_dev.loc[8]['ID']) > int(spmrl_dev.loc[9]['ID'])

In [15]:
sentence_id(ud_dev, 'ud')

In [ ]:
ud_dev[ud_dev['FEATS'].str.contains('Fut', na=False)]['FEATS'].unique()

In [ ]:
for i, row in spmrl_dev.iterrows():
    print(type(row['ID']))

In [ ]:
spmrl_dev = suit_for_pandas(filepath_spmrl_dev)

In [ ]:
spmrl_train = suit_for_pandas(filepath_spmrl_train)

In [ ]:
spmrl_test = suit_for_pandas(filepath_spmrl_test)

In [ ]:
spmrl_dev[spmrl_dev['sent_id'] == 99]

In [ ]:
spmrl_dev[spmrl_dev['XPOS'].str.contains('S_PRN', na=False)]#['FEATS'].unique()

In [ ]:
ud_dev = suit_for_pandas('./data/he_htb-ud-dev.conllu')
ud_dev.columns = ['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC']

In [ ]:
spmrl_dev[spmrl_dev['XPOS'].str.contains('NN_S_PP', na=False)]

In [ ]:
ud_dev[(ud_dev['XPOS'].str.contains('BN', na=False)) & (ud_dev['FEATS'].str.contains('HIFIL', na=False))]

In [ ]:
spmrl_dev.to_csv('./data/spmrl-treebank/numbered_dev.csv')